# Import Library & Setup

In [12]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, callbacks
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization

# ML
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    silhouette_score, davies_bouldin_score, calinski_harabasz_score,
    adjusted_rand_score
)
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.ensemble import RandomForestClassifier

# Viz
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

np.random.seed(42)
tf.random.set_seed(42)

print('Libraries loaded')
print(f'Time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

Libraries loaded
Time: 2026-02-06 11:20:31


# Load Data

In [13]:
file = '../data/trail_dataset.csv'

try:
    df = pd.read_csv(file)
    print(f'Loaded: {df.shape[0]} shoes × {df.shape[1]} columns')
    display(df.head())
except FileNotFoundError:
    print(f"WARNING: '{file}' not found.")
    print("Please upload the correct dataset file to run with actual data.")

Loaded: 158 shoes × 57 columns


,brand,name,terrain_light,terrain_moderate,terrain_technical,shock_low,shock_moderate,shock_high,energy_low,energy_moderate,...,forefoot_lab_mm,season_summer,season_winter,season_all,heavy_runners,removable_insole,orthotic_friendly,waterproof,water_repellent,lightweight
0,adidas,terrex agravic speed ultra,1,0,0,0,1,0,0,0,...,30.3,0,0,1,0,1,1,0,0,0
1,adidas,terrex speed ultra,1,0,0,0,0,0,0,0,...,24.6,0,0,0,0,1,1,0,0,0
2,altra,experience wild,1,1,0,0,1,0,1,0,...,30.2,0,0,1,0,1,1,0,0,0
3,altra,experience wild 2,1,0,0,0,1,0,1,0,...,26.2,0,0,1,0,1,1,0,0,0
4,altra,lone peak 5.0,1,1,0,0,0,0,0,0,...,24.3,0,0,0,0,1,1,0,0,0


In [14]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 57 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand                158 non-null    str    
 1   name                 158 non-null    str    
 2   terrain_light        158 non-null    int64  
 3   terrain_moderate     158 non-null    int64  
 4   terrain_technical    158 non-null    int64  
 5   shock_low            158 non-null    int64  
 6   shock_moderate       158 non-null    int64  
 7   shock_high           158 non-null    int64  
 8   energy_low           158 non-null    int64  
 9   energy_moderate      158 non-null    int64  
 10  energy_high          158 non-null    int64  
 11  traction_moderate    158 non-null    int64  
 12  traction_high        158 non-null    int64  
 13  arch_neutral         158 non-null    int64  
 14  arch_stability       158 non-null    int64  
 15  weight_lab_oz        158 non-null    float64
 16  d

# Preprocessing

In [15]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

binary_cols = [col for col in numeric_cols if set(df[col].unique()).issubset({0, 1})]
continuous_cols = [col for col in numeric_cols if col not in binary_cols]

print(f'Features: {len(numeric_cols)} total')
print(f'  Binary     : {len(binary_cols)}')
print(f'  Continuous : {len(continuous_cols)}')

Features: 55 total
  Binary     : 46
  Continuous : 9


In [16]:
feature_cols = numeric_cols.copy()
X = df[feature_cols]

# Separate for proper scaling
X_binary = X[binary_cols].values
X_continuous = X[continuous_cols].values

# Scale continuous to 0-1 for neural network
scaler_continuous = MinMaxScaler()
X_continuous_scaled = scaler_continuous.fit_transform(X_continuous)

# Combine
X_combined = np.concatenate([X_binary, X_continuous_scaled], axis=1)

# Also standard scaling for traditional comparison
scaler_standard = StandardScaler()
X_standard = scaler_standard.fit_transform(X)

print(f'Neural input shape: {X_combined.shape}')
print(f'Range: [{X_combined.min():.6f}, {X_combined.max():.6f}]')

Neural input shape: (158, 55)
Range: [0.000000, 1.000000]


# Auto-Encoder

## Modelling

In [17]:
# Architecture
input_dim = X_combined.shape[1]
encoding_dims = [32, 16, 8]

# Encoder
input_layer = Input(shape=(input_dim,))
x = input_layer
for dim in encoding_dims:
    x = Dense(dim, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

latent = x

# Decoder
for dim in reversed(encoding_dims[:-1]):
    x = Dense(dim, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

output_layer = Dense(input_dim, activation='sigmoid')(x)

autoencoder = Model(input_layer, output_layer)
encoder = Model(input_layer, latent)

autoencoder.compile(
    optimizer=optimizers.Adam(0.001),
    loss='mse',
    metrics=['mae']
)

print('Autoencoder architecture:')
autoencoder.summary()

Autoencoder architecture:


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 55)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 8)              │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 55)             │         1,815 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,375 (21.00 KB)

 Trainable params: 5,167 (20.18 KB)

 Non-trainable params: 208 (832.00 B)

## Training

In [18]:
history = autoencoder.fit(
    X_combined, X_combined,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
        callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-5)
    ],
    verbose=0
)

print(f'Training done!')
print(f'Final loss: {history.history["loss"][-1]:.6f}')
print(f'  Val loss: {history.history["val_loss"][-1]:.6f}')

# Get latent representations
X_latent = encoder.predict(X_combined, verbose=0)
print(f'Latent space: {X_latent.shape} (8D embeddings)')

Training done!
Final loss: 0.097321
  Val loss: 0.109388
Latent space: (158, 8) (8D embeddings)


# Metrics Function

In [19]:
def calculate_cluster_purity(df, cluster_col, binary_cols):
    purity_by_cluster = {}
    for cid in df[cluster_col].unique():
        cdata = df[df[cluster_col] == cid]
        n = len(cdata)
        dominances = []
        for col in binary_cols:
            if col in cdata.columns:
                vc = cdata[col].value_counts()
                if len(vc) > 0:
                    dominances.append(vc.max() / n)
        purity_by_cluster[cid] = {'purity': np.mean(dominances) if dominances else 0, 'n': n}
    all_p = [v['purity'] for v in purity_by_cluster.values()]
    return {
        'by_cluster': purity_by_cluster,
        'mean_purity': np.mean(all_p),
        'min_purity': np.min(all_p),
        'max_purity': np.max(all_p)
    }

def calculate_cluster_stability(X, labels, model_func, n_iter=20):
    n = len(X)
    ari_scores = []
    for _ in range(n_iter):
        idx = np.random.choice(n, n, replace=True)
        boot_labels = model_func().fit_predict(X[idx])
        ari = adjusted_rand_score(labels[idx], boot_labels)
        ari_scores.append(ari)
    m = np.mean(ari_scores)
    return {
        'mean_ari': m,
        'std_ari': np.std(ari_scores),
        'stability_level': 'Excellent' if m > 0.8 else 'Good' if m > 0.6 else 'Moderate'
    }

def calculate_interpretability_score(df, cluster_col, binary_cols, threshold=0.75):
    scores = []
    for cid in df[cluster_col].unique():
        cdata = df[df[cluster_col] == cid]
        n = len(cdata)
        strong = sum(1 for col in binary_cols if col in cdata.columns and
                    (cdata[col].sum()/n > threshold or cdata[col].sum()/n < 1-threshold))
        scores.append(strong / len(binary_cols))
    return {'mean_interpretability': np.mean(scores), 'scores': scores}

def evaluate_clustering_comprehensive(X, labels, df_temp, model_func, binary_cols):
    sil = silhouette_score(X, labels)
    db = davies_bouldin_score(X, labels)
    ch = calinski_harabasz_score(X, labels)
    df_temp['cluster'] = labels
    purity = calculate_cluster_purity(df_temp, 'cluster', binary_cols)
    stability = calculate_cluster_stability(X, labels, model_func, 10)
    interp = calculate_interpretability_score(df_temp, 'cluster', binary_cols)

    sil_norm = (sil + 1) / 2
    db_norm = 1 / (1 + db)
    ch_norm = min(ch / 1000, 1)
    composite = (0.25*sil_norm + 0.20*db_norm + 0.15*ch_norm +
                 0.25*purity['mean_purity'] + 0.10*stability['mean_ari'] +
                 0.05*interp['mean_interpretability'])

    return {
        'silhouette': sil, 'davies_bouldin': db, 'calinski_harabasz': ch,
        'purity': purity['mean_purity'], 'stability': stability['mean_ari'],
        'interpretability': interp['mean_interpretability'], 'composite_score': composite
    }

print('Metrics functions ready')

Metrics functions ready


# Model Selection

In [20]:
results = []

# Header Tabel
print(f"| {'K':^3} | {'Score':^8} | {'Sil.':^8} | {'DB':^8} | {'CH':^9} | {'Purity':^8} | {'Stab.':^8} | {'Interp':^8} |")
print(f"|{'-'*5}+{'-'*10}+{'-'*10}+{'-'*10}+{'-'*11}+{'-'*10}+{'-'*10}+{'-'*10}|")

for i in range(3, 10):
    np.random.seed(42)
    
    model_factory = lambda: KMeans(n_clusters=i, random_state=42, n_init=10)
    model = model_factory()
    labels = model.fit_predict(X_latent)

    metrics = evaluate_clustering_comprehensive(
        X_latent, labels, df.copy(),
        model_factory,
        binary_cols
    )

    # Simpan hasil
    results.append({
        'k': i,
        'model': model,
        'labels': labels,
        **metrics
    })

    # Print Baris Tabel
    print(f"| {i:^3} | {metrics['composite_score']:<8.6f} | {metrics['silhouette']:<6.6f} | "
          f"{metrics['davies_bouldin']:<6.6f} | {metrics['calinski_harabasz']:<8.6f} | "
          f"{metrics['purity']:<6.6f} | {metrics['stability']:<6.6f} | {metrics['interpretability']:<6.6f} |")

|  K  |  Score   |   Sil.   |    DB    |    CH     |  Purity  |  Stab.   |  Interp  |
|-----+----------+----------+----------+-----------+----------+----------+----------|
|  3  | 0.568847 | 0.303009 | 1.379291 | 50.824194 | 0.821396 | 0.745198 | 0.688406 |
|  4  | 0.579926 | 0.297565 | 1.211280 | 52.941533 | 0.836309 | 0.757556 | 0.690217 |
|  5  | 0.573495 | 0.295685 | 1.201418 | 50.444052 | 0.840634 | 0.655672 | 0.747826 |
|  6  | 0.565409 | 0.270759 | 1.149318 | 49.182558 | 0.842977 | 0.586147 | 0.735507 |
|  7  | 0.569439 | 0.269324 | 1.159836 | 49.809705 | 0.851218 | 0.607864 | 0.742236 |
|  8  | 0.575248 | 0.282745 | 1.144349 | 48.904090 | 0.850782 | 0.645128 | 0.741848 |
|  9  | 0.574356 | 0.275402 | 1.125322 | 47.061565 | 0.859817 | 0.619780 | 0.736715 |


In [21]:
df_results = pd.DataFrame(results)
best_config = df_results.loc[df_results['composite_score'].idxmax()]

best_model = best_config['model']
best_labels = best_config['labels']
best_k = best_config['k']
X_for_clustering = X_latent

print(f'SELECTED BEST K: {best_k}')
print(f'   Silhouette      : {best_config["silhouette"]:.6f}')
print(f'   Composite Score : {best_config["composite_score"]:.6f}')

SELECTED BEST K: 4
   Silhouette      : 0.297565
   Composite Score : 0.579926


# Generate Cluster Labels

In [ ]:
# Masukkan Cluster ke DataFrame
df['cluster'] = best_labels 

# Setup Grouping
bin_groups = {}
for col in binary_cols:
    parts = col.split('_')
    
    if len(parts) > 1:
        prefix = '_'.join(parts[:-1])
    else:
        prefix = col
        
    bin_groups.setdefault(prefix, []).append(col)

# Build Summary Data
rows = []
for cid in sorted(df['cluster'].unique()):
    subset = df[df['cluster'] == cid]
    n = len(subset)
    
    row = {'count': n, 'percentage': f"{n/len(df)*100:.1f}%"}

    # A. Continuous Columns: Langsung ambil mean
    for col in continuous_cols:
        row[col.lower()] = round(subset[col].mean(), 2)

    # B. Binary Groups
    for prefix, cols in bin_groups.items():
        # Hitung mean grup ini
        means = subset[cols].mean()
        best_col = means.idxmax()
        best_val = means.max()
        
        # Case 1: Multiple Variants
        if len(cols) > 1:
            header = prefix.lower()
            val_str = best_col.replace(f"{prefix}_", "").lower()
            row[header] = f"{val_str} ({best_val*100:.0f}%)"
            
        # Case 2: Standalone
        else:
            header = cols[0].lower()
            val_str = "yes" if best_val > 0.5 else "no"
            row[header] = f"{val_str} ({best_val*100:.0f}%)"

    rows.append(row)

# Create DataFrame & Fix Display
df_summary = pd.DataFrame(rows, index=sorted(df['cluster'].unique()))
df_summary.index.name = None 

print("Cluster Summary:")
display(df_summary)

Cluster Summary:


,count,percentage,weight_lab_oz,drop_lab_mm,toebox_durability,heel_durability,outsole_durability,breathability,lug_depth,heel_lab_mm,...,stiffness,torsional,heel_stiff,season,heavy_runners,removable_insole,orthotic_friendly,waterproof,water_repellent,lightweight
0,24,15.2%,10.15,5.87,0.17,0.00,0.00,1.08,3.66,30.35,...,stiff (100%),stiff (29%),moderate (29%),all (33%),no (0%),yes (75%),yes (75%),no (0%),no (4%),no (8%)
1,32,20.3%,9.96,6.37,3.03,3.12,3.25,2.78,3.69,33.02,...,moderate (91%),stiff (47%),flexible (50%),all (94%),no (3%),yes (91%),yes (91%),no (3%),no (3%),no (12%)
2,80,50.6%,10.46,9.52,2.81,2.88,3.11,2.92,3.48,35.55,...,stiff (75%),stiff (88%),stiff (49%),all (81%),no (8%),yes (100%),yes (100%),no (14%),no (4%),no (2%)
3,22,13.9%,9.59,3.32,2.55,2.77,3.23,3.14,3.07,24.99,...,stiff (59%),moderate (50%),flexible (73%),all (91%),no (0%),yes (91%),yes (91%),no (5%),no (5%),no (14%)
